In [54]:
import pandas as pd
import numpy as np
import joblib
import zipfile
import os

### Loading data

In [81]:
# Load the model
consumption = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/train.csv') # Replace with "pickle.load" if you used pickle
building = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/building_metadata.csv')
weather = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/weather_train.csv')


In [64]:
len(consumption[consumption['meter_reading']<= 0])/len(consumption[consumption['meter'] == 0])

0.1553760039665332

In [82]:
consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [66]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.1+ KB


In [67]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


### Filtering consumption only for valid electrictiy readings (meter = 0)

In [68]:
electricity = consumption[(consumption['meter'] == 0) & (consumption['meter_reading']> 0)]

In [69]:
electricity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11530741 entries, 45 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 439.9+ MB


### Merging everything into two datasets

In [70]:
electricity_building = electricity.merge(building, on='building_id', how='left')


electricity_building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11530741 entries, 0 to 11530740
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
 4   site_id        int64  
 5   primary_use    object 
 6   square_feet    int64  
 7   year_built     float64
 8   floor_count    float64
dtypes: float64(3), int64(4), object(2)
memory usage: 791.8+ MB


In [71]:
electricity_building_weather = electricity_building.merge(weather, on=['site_id', 'timestamp'], how='left')


electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11530741 entries, 0 to 11530740
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int64  
 1   meter               int64  
 2   timestamp           object 
 3   meter_reading       float64
 4   site_id             int64  
 5   primary_use         object 
 6   square_feet         int64  
 7   year_built          float64
 8   floor_count         float64
 9   air_temperature     float64
 10  cloud_coverage      float64
 11  dew_temperature     float64
 12  precip_depth_1_hr   float64
 13  sea_level_pressure  float64
 14  wind_direction      float64
 15  wind_speed          float64
dtypes: float64(10), int64(4), object(2)
memory usage: 1.4+ GB


### Saving merged datasets

In [80]:
# Specify the zip file name
zip_filename = "../dataset/filtered.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "filtered.pkl"
    joblib.dump(electricity_building_weather, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="filtered.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)
